In [2]:
"""
Reflector : A
Rotore : I-II-III
Plugboard: A-R, G-K, O-X
Message: A=> X
"""

"""
I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")


"""
class Keyboard:
    
    def forward(self, letter):       # converts letter to signal
        signal = "ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(letter)
        return signal
    def backward(self, signal):
        letter="ABCDEFGHIJKLMNOPQRSTUVWXYZ"[signal]
        return letter
        
        



In [3]:
class Plugboard:
    def __init__(self,pairs):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for pair in pairs:
            A=pair[0]
            B=pair[1]
            pos_A=self.left.find(A)
            pos_B=self.left.find(B)
            self.left=self.left[:pos_A] + B + self.left[pos_A+1:]     # swap for plugboard
            self.left=self.left[:pos_B] + A + self.left[pos_B+1:]
            
    def forward(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal
    
    def backward(self,signal):
        letter=self.left[signal]
        signal=self.right.find(letter)
        return signal


            
            
    
    

In [4]:
class Rotor:
    def __init__(self, wiring, notch):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right=wiring
        self.notch=notch
        
    def forward(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal
    
    def backward(self,signal):
        letter=self.left[signal]
        signal=self.right.find(letter)
        return signal
    
    def show(self):              # show rotor?
        print(self.left)
        print(self.right)
        print("")
        
    
    def rotate(self,n=1, forward=True):# rings part no need forward=true/else
        for i in range(n):
            if forward:
                self.left=self.left[1:]+self.left[0]
                self.right=self.right[1:]+self.right[0]
            else:
                self.left=self.left[25]+self.left[:25]
                self.right=self.right[25]+self.right[:25]
                
                
        
    def rotate_to_letter(self,letter):
        n = "ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(letter)
        self.rotate(n)
        
    
    def set_ring(self,n):          # ring part
        # rotate the rotor backwards
            self.rotate(n-1, forward=False)
        # adjust the turnover notch in relationship to the wiring
            n_notch="ABCDEFGHIJKLMNOPQRSTUVWXYZ".find(self.notch)
            self.notch="ABCDEFGHIJKLMNOPQRSTUVWXYZ"[(n_notch-n)%26]
            
        



In [5]:
class Reflector:
    def __init__(self, wiring,):
        self.left="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.right=wiring
        
    def reflect(self,signal):
        letter=self.right[signal]
        signal=self.left.find(letter)
        return signal
    
   

In [6]:
class Enigma:                                  # rerun every time?
    def __init__(self,re,r1,r2,r3,pb,kb):
        self.re=re
        self.r1=r1
        self.r2=r2
        self.r3=r3
        self.pb=pb
        self.kb=kb
    def set_key(self,key):
        self.r1.rotate_to_letter(key[0])
        self.r2.rotate_to_letter(key[1])
        self.r3.rotate_to_letter(key[2])
        
    def set_rings(self, rings):             # ring part
        self.r1.set_ring(rings[0])
        self.r2.set_ring(rings[1])
        self.r3.set_ring(rings[2])
        
        
        
    def encipher(self,letter):
        if self.r2.left[0]==self.r2.notch and self.r3.left[0]==self.r3.notch:
            self.r1.rotate()
            self.r2.rotate()
            self.r3.rotate()
        elif self.r2.left[0]==self.r2.notch: # double step anomaly
            self.r1.rotate()
            self.r2.rotate()
            self.r3.rotate()
            
        elif self.r3.left[0]==self.r3.notch:
            self.r2.rotate()
            self.r3.rotate()
        else:
            self.r3.rotate()
            
       
        
       
        
        
        signal=self.kb.forward(letter)
        signal=self.pb.forward(signal)
        signal=self.r3.forward(signal)
        signal=self.r2.forward(signal)
        signal=self.r1.forward(signal)
        signal=self.re.reflect(signal)
        signal=self.r1.backward(signal)
        signal=self.r2.backward(signal)
        signal=self.r3.backward(signal)
        signal=self.pb.backward(signal)
        letter=self.kb.backward(signal)
        return letter

In [7]:
#historical enigma components
I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")
C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")

KB=Keyboard()
PB=Plugboard(["AB","CD","EF"])




ENIGMA=Enigma(A,I,II,III,PB,KB)

# set the rings
ENIGMA.set_rings((1,1,1))


ENIGMA.set_key("ZZZ")  




message="THISISATESTTOENCRYPTAPARAGRAPHOFDATASUCCESSFULLYBUTTHISISALINENOTAPARAGRAPHITSFINE"
cipher_text=""
for letter in message:
    cipher_text=cipher_text + ENIGMA.encipher(letter)
print(cipher_text)


print(ENIGMA.encipher("A"))






NMDNAXZIBJEYYWYIERCDXYTNISQHUIBJYSCXBKGGRJIYORCIVYQVMNNQKENEBHLHOJBXMHENOYJWLNYJAP
M


In [8]:
# cracking the Enigma basic approach

import time
h1=time.time()
 
message="NMDNAXZIBJEYYWYIERCDXYTNISQHUIBJYSCXBKGGRJIYORCIVYQVMNNQKENEBHLHOJBXMHENOYJWLNYJAP"

alphabet="ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# finding out which key is the correct one

for x in range(0,26):
    for y in range(0,26):
        for z in range(0,26):
            I=Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ","Q")          # taken from Wikipedia
            II=Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE","E")
            III=Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO","V")
            IV=Rotor("ESOVPZJAYQUIRHXLNFTGKDCMWB","J")
            V=Rotor("VZBRGITYUPSDNHLXAWMJQOFECK","Z")
            A=Reflector("EJMZALYXVBWFCRQUONTSPIKHGD")
            B=Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")          # re-initializing enigma to ground state each iteration
            C=Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")

            KB=Keyboard()
            PB=Plugboard(["AB","CD","EF"])




            ENIGMA=Enigma(A,I,II,III,PB,KB)

            # set the rings
            ENIGMA.set_rings((1,1,1))
            key=(alphabet[x]+alphabet[y]+alphabet[z])
            ENIGMA.set_key(key)
            cipher_text=""
            for letter in message:
                cipher_text=cipher_text + ENIGMA.encipher(letter)
                if cipher_text=="THISISATESTTOENCRYPTAPARAGRAPHOFDATASUCCESSFULLYBUTTHISISALINENOTAPARAGRAPHITSFINE":
                    print("key is",key)
                    break
h2=time.time()
print("time taken is",h2-h1)
            







key is ZZZ
time taken is 9.631562232971191
